In [3]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from pydub import AudioSegment
import os
import subprocess
import textwrap

# === 设置参数 ===
TEXT = """亲爱的女儿：
    20年号，你来到了爸妈的身边，到今天为止你已经一周岁啦。
    在你的记忆里，这一年也许都是空白的，
    但是这一年却是大家最繁忙，最珍贵的一年。
    因为你的到来，改变了很多事情。
    你一定会马不停蹄的长大，此后爸妈一定会怀念此时小小的你。
     
     希望你是我身边的一颗小树，终是长成自己的模样
     终有一天与我比肩而立，却有着自己的思想
     希望你如天上的星辰，与吾相似却不尽然
     期待你在属于自己的宇宙中，熠熠生辉，自由闪耀。
          
     轻舟已过万重山，前路虽然漫漫。
     成长之路坎坷艰辛，你会体会生活的酸甜苦辣。
     愿你能坚强与勇敢，心里有爱，眼里有光。
     愿你在爱中学会温柔，在风雨中学会坚定。 
     愿你拥有向阳的力量，也有拥抱世界的勇气。"""
FONT_PATH = "MaShanZheng-Regular.ttf"
FONT_SIZE = 40  # 稍微增大字体以适应16:9屏幕
VIDEO_SIZE = (1920, 1080)  # 标准16:9分辨率
TEXT_COLOR = (255, 255, 255)
BG_COLOR = (0, 0, 0)
FPS = 24
CHAR_DELAY = 2  # 每个字符的延迟帧数
END_PAUSE_SECONDS = 8  # 视频结束时的停顿时间（秒）
OUTPUT_VIDEO = "typing_video.avi"
OUTPUT_FINAL = "typing_final_with_audio.mp4"
TYPE_SOUND = "type_sound.mp3"  # 确保此文件存在

# 16:9屏幕的文本布局参数
TEXT_MARGIN_LEFT = 160  # 左边距
TEXT_MARGIN_TOP = 120   # 上边距
TEXT_WIDTH = 45         # 每行大约字符数
LINE_SPACING = 1.5      # 行间距倍数

# 验证文件存在
if not os.path.exists(TYPE_SOUND):
    print(f"警告: 音频文件 {TYPE_SOUND} 不存在!")
    exit(1)
else:
    print(f"找到音频文件: {TYPE_SOUND}")

# === 处理文本以适应16:9屏幕 ===
def format_text_for_display(text):
    # 按原始段落分割
    paragraphs = text.split('\n')
    formatted_paragraphs = []
    
    for p in paragraphs:
        # 如果段落以空格开头（缩进），保留一个空格的缩进
        if p.startswith(' '):
            indent = '    '
            p = p.lstrip()
        else:
            indent = ''
        
        # 处理每个段落，限制行宽
        if p.strip():  # 跳过空行
            wrapped = textwrap.fill(p.strip(), width=TEXT_WIDTH)
            # 添加缩进到每一行
            indented = '\n'.join(indent + line for line in wrapped.split('\n'))
            formatted_paragraphs.append(indented)
        else:
            formatted_paragraphs.append('')  # 保留空行
    
    return '\n'.join(formatted_paragraphs)

# 格式化文本
formatted_text = format_text_for_display(TEXT)
print("文本已格式化为16:9屏幕显示")

# === 生成帧 ===
try:
    font = ImageFont.truetype(FONT_PATH, FONT_SIZE)
    frames = []
    text_so_far = ""
    
    # 定义需要停顿的文本段落结束标记
    pause_points = [
        "但是这一年却是大家最繁忙，最珍贵的一年。",
        "你一定会马不停蹄的长大，此后爸妈一定会怀念此时小小的你。",
        "期待你在属于自己的宇宙中，熠熠生辉，自由闪耀。"
    ]
    pause_frames = 48  # 停顿2秒 (24fps × 2)

    # 提前计算行高以便于定位
    line_height = int(FONT_SIZE * LINE_SPACING)
    
    for char in TEXT:
        text_so_far += char
        current_delay = CHAR_DELAY
        
        # 检查是否需要在当前位置停顿
        for pause_text in pause_points:
            if text_so_far.endswith(pause_text):
                print(f"在文本 '{pause_text}' 后添加停顿")
                current_delay += pause_frames
                break
                
        for _ in range(current_delay):
            img = Image.new("RGB", VIDEO_SIZE, BG_COLOR)
            draw = ImageDraw.Draw(img)
            
            # 处理文本显示（带格式）
            formatted_so_far = format_text_for_display(text_so_far)
            lines = formatted_so_far.split('\n')
            
            y = TEXT_MARGIN_TOP
            for line in lines:
                draw.text((TEXT_MARGIN_LEFT, y), line, font=font, fill=TEXT_COLOR)
                y += line_height
            
            frame = np.array(img)
            frames.append(frame)

    # 添加视频末尾的停顿
    end_pause_frames = int(END_PAUSE_SECONDS * FPS)
    print(f"在视频末尾添加 {END_PAUSE_SECONDS} 秒停顿 ({end_pause_frames} 帧)")
    
    # 创建包含完整文本的最终帧
    final_img = Image.new("RGB", VIDEO_SIZE, BG_COLOR)
    final_draw = ImageDraw.Draw(final_img)
    
    # 显示格式化的完整文本
    formatted_full = format_text_for_display(TEXT)
    lines = formatted_full.split('\n')
    
    y = TEXT_MARGIN_TOP
    for line in lines:
        final_draw.text((TEXT_MARGIN_LEFT, y), line, font=font, fill=TEXT_COLOR)
        y += line_height
        
    final_frame = np.array(final_img)
    
    # 重复最终帧以创建停顿效果
    for _ in range(end_pause_frames):
        frames.append(final_frame)

    print(f"总共生成了 {len(frames)} 帧视频")

    # === 写入视频 ===
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, FPS, VIDEO_SIZE)

    for frame in frames:
        video.write(frame)
    video.release()
    print(f"视频已保存到: {OUTPUT_VIDEO}")

    # === 生成音频（打字声拼接） ===
    try:
        base_sound = AudioSegment.from_file(TYPE_SOUND)
        typed_audio = AudioSegment.silent(duration=0)
        
        # 根据文本生成对应的打字声
        current_text = ""
        for char in TEXT:
            current_text += char
            
            # 添加打字声
            typed_audio += base_sound[:150]
            typed_audio += AudioSegment.silent(duration=100)
            
            # 检查是否需要在当前位置停顿
            for pause_text in pause_points:
                if current_text.endswith(pause_text):
                    # 添加额外的静音来匹配视频的停顿
                    pause_duration = pause_frames * 1000 // FPS  # 将帧数转换为毫秒
                    typed_audio += AudioSegment.silent(duration=pause_duration)
                    break

        # 添加末尾停顿的静音
        end_pause_duration = END_PAUSE_SECONDS * 1000  # 转换为毫秒
        typed_audio += AudioSegment.silent(duration=end_pause_duration)
        
        # 保存临时音频
        temp_audio = "typing_audio.mp3"
        typed_audio.export(temp_audio, format="mp3")
        print(f"音频已生成: {temp_audio}")

        # === 合成音视频（使用 ffmpeg）===
        cmd = [
            "ffmpeg", "-y", 
            "-i", OUTPUT_VIDEO, 
            "-i", temp_audio, 
            "-c:v", "libx264", 
            "-preset", "medium",
            "-crf", "22",
            "-c:a", "aac", 
            "-strict", "experimental",
            OUTPUT_FINAL
        ]
        
        print("执行合成命令:", " ".join(cmd))
        subprocess.run(cmd, check=True)
        print(f"最终视频已生成: {OUTPUT_FINAL}")
        
        # # 清理临时文件
        # if os.path.exists(temp_audio):
        #     os.remove(temp_audio)
        # if os.path.exists(OUTPUT_VIDEO):
        #     os.remove(OUTPUT_VIDEO)
        
    except Exception as e:
        print(f"处理音频时出错: {e}")

except Exception as e:
    print(f"发生错误: {e}")

找到音频文件: type_sound.mp3
文本已格式化为16:9屏幕显示
在文本 '但是这一年却是大家最繁忙，最珍贵的一年。' 后添加停顿
在文本 '你一定会马不停蹄的长大，此后爸妈一定会怀念此时小小的你。' 后添加停顿
在文本 '期待你在属于自己的宇宙中，熠熠生辉，自由闪耀。' 后添加停顿
在视频末尾添加 8 秒停顿 (192 帧)
总共生成了 1124 帧视频
视频已保存到: typing_video.avi
音频已生成: typing_audio.mp3
执行合成命令: ffmpeg -y -i typing_video.avi -i typing_audio.mp3 -c:v libx264 -preset medium -crf 22 -c:a aac -strict experimental typing_final_with_audio.mp4
最终视频已生成: typing_final_with_audio.mp4
